<a href="https://colab.research.google.com/github/Ramubala/face-recognition/blob/main/FaceVerification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !rm -rf '/content/data' 
# !rm -rf '/content/annotations'

In [ ]:
# !mkdir data
# !mkdir annotations
# !tar zxvf '/content/drive/MyDrive/datasets/originalPics.tar.gz' -C '/content/data/' ;
# !tar zxvf '/content/drive/MyDrive/datasets/FDDB-folds.tgz' -C '/content/annotations/';

In [ ]:
# import glob
# file_list = [file for file in glob.glob(r'/content/annotations/FDDB-folds/*-ellipseList.txt',recursive=True)]
# train_files, test_files = file_list[:9],[file_list[9]]
# print(train_files, test_files)

In [ ]:
#custom_train_dataset[2]

In [ ]:
# def per_person_stats(source_txt: str):
#   stats_dict=dict()
#   with open(source_txt,'r') as f:
#     data = f.readlines()
#     for i in range(1, len(data)):
#       line = data[i].strip().split("\t")
#       stats_dict[line[0]] = int(line[1])
#   f.close()
#   return stats_dict

In [ ]:
# def pick_train_pairs(_dict, num_per_sample, num_negative_faces_per_sample):
#   if num_per_sample!=3:
#     raise Exception("only 3 per sample supported by algorithm")
#   allfaces = list(_dict.keys())
#   data = []
#   for key in allfaces:
#     # anchor and postive example
#     facepath = '/content/data/lfw-py/lfw_funneled/{}'.format(key)
#     available_images = [file for file in glob.glob(facepath+'/*.jpg')]
#     num_available_images = _dict[key]
#     num_achor_postive_pairs = num_available_images*(num_available_images-1)
#     if num_achor_postive_pairs==0:
#       combinations = [(available_images[0],available_images[0])]
#     else:
#       combinations = list(itertools.combinations(available_images, 2))

#     # negative face
#     possible_choices = [v for v in allfaces if v != key]
#     negative_examples = random.sample(possible_choices, num_negative_faces_per_sample)
#     for negative_example in negative_examples:
#       negfacepath = '/content/data/lfw-py/lfw_funneled/{}'.format(negative_example)
#       available_images = [file for file in glob.glob(negfacepath+'/*.jpg')]
#       neg_imgpath = random.choice(available_images)

#       for tuple_el in combinations:
#         data.append((tuple_el[0],tuple_el[1],neg_imgpath))
#   return data  

In [ ]:
# train_txt_file = '/content/data/lfw-py/peopleDevTrain.txt'
# test_txt_file = '/content/data/lfw-py/peopleDevTest.txt'
# train_stats = per_person_stats(train_txt_file)
# test_stats = per_person_stats(test_txt_file)

In [ ]:
#test_stats.keys() & train_stats.keys()

In [ ]:
# train_data = pick_train_pairs(train_stats, 3, 5)

In [ ]:
# !pip install -q condacolab
# import condacolab
# condacolab.install()
# !conda install cudnn=8.3.2

# VERSION = "20200318" #@param ["20200318","nightly", "xrt==1.15.0"]
# !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --version $VERSION

In [1]:
import torchvision
import torch
import os
import glob
import numpy as np
import itertools
import random
random.seed(42)
from torch.utils.data import DataLoader,Dataset,Sampler
import torchvision.transforms as transforms
# import torch_xla
# import torch_xla.core.xla_model as xm

In [2]:
#train_dataset = torchvision.datasets.LFWPeople('/content/data/',split='train',download=True)
#test_dataset = torchvision.datasets.LFWPeople('/content/data/',split='test',download=True)

#train_dataset = torchvision.datasets.MNIST('/content/data/',train=True,download=True)
#test_dataset = torchvision.datasets.MNIST('/content/data/',train=False,download=True)
mean, std = 0.28604059698879553, 0.35302424451492237

train_dataset = torchvision.datasets.FashionMNIST('./content/data/FashionMNIST', train=True, download=True,
                             transform=transforms.Compose([
                                 transforms.ToTensor(),
                                 transforms.Normalize((mean,), (std,))
                             ]))
test_dataset = torchvision.datasets.FashionMNIST('/content/data/FashionMNIST', train=False, download=True,
                            transform=transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((mean,), (std,))
                            ]))

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./content/data/FashionMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./content/data/FashionMNIST/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./content/data/FashionMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./content/data/FashionMNIST/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

...

Extracting /content/data/FashionMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /content/data/FashionMNIST/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting /content/data/FashionMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /content/data/FashionMNIST/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting /content/data/FashionMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /content/data/FashionMNIST/FashionMNIST/raw



In [3]:
len(train_dataset), len(test_dataset)

(60000, 10000)

In [4]:
from torch.utils.data.dataloader import BatchSampler
import gc

class CustomDataset(Dataset):
  def __init__(self, src_dataset, upsampling_factor):
    self.src_dataset = src_dataset
    self.src_labels = np.array([src_dataset[i][1] for i in range(len(src_dataset))])
    self.labels = list(set(self.src_labels))
    self.label_to_index= {label: np.where(self.src_labels == label)[0]
                                     for label in self.labels}
    self.upsampling_factor = upsampling_factor
    self.transform = transforms.Compose([
          transforms.ToTensor()
          ])
    
  def __len__(self):
      return self.upsampling_factor*len(self.src_dataset)

  def __getitem__(self, idx : int):
      # anchor image
      ref_dataset_index = int(idx/self.upsampling_factor)
      ref_train_image = self.src_dataset[ref_dataset_index]
      label = ref_train_image[1]

      # finding positive example
      possible_positive_choices = self.label_to_index[label]
      possible_positive_choices = [v for v in possible_positive_choices if v!= idx]
      choice = random.choice(possible_positive_choices)
      positive_train_image = self.src_dataset[choice]

      # finding negative example
      possible_neg_choices = [v for v in self.labels if v!= label]
      choice = random.choice(possible_neg_choices)
      choose_neg_idx = random.choice(self.label_to_index[choice])
      negative_train_image = self.src_dataset[choose_neg_idx]

      return (ref_train_image[0], positive_train_image[0], negative_train_image[0]), (ref_train_image[1], positive_train_image[1], negative_train_image[1])


_train_dataset = CustomDataset(train_dataset, 1)
_test_dataset = CustomDataset(test_dataset, 1)

In [5]:
len(_train_dataset), len(_test_dataset)

(60000, 10000)

In [ ]:
# torch.save(_train_dataset, 'train.pt')
# torch.save(_test_dataset,'test.pt')

In [6]:
class BatchSamplerUniform(Sampler):
  def __init__(self, labels, n_classes, n_samples):
    self.labels = labels
    self.batch_size = n_classes*n_samples
    self.unique_labels = list(set(labels))
    self.n_classes = n_classes
    self.n_samples = n_samples
    self.labels_to_index_map = {label: np.where(self.labels == label)[0]
                                      for label in self.unique_labels}
    self.used_up_samples = {label: 0 for label in self.unique_labels}
    self.count=0

  def __iter__(self):
    self.count=0
    while self.count + self.batch_size < len(self.labels):
      sampled_classes = np.random.choice(self.unique_labels, self.n_classes, replace=False)
      indices = []
      for label in sampled_classes:
        available_indices = self.labels_to_index_map[label]
        last_index_used = self.used_up_samples[label]
        samples = available_indices[last_index_used:last_index_used+self.n_samples]
        self.used_up_samples[label] = last_index_used+self.n_samples
        if self.used_up_samples[label]+self.n_samples>len(available_indices):
          np.random.shuffle(self.labels_to_index_map[label])
          self.used_up_samples[label]=0
        indices.extend(samples)
      self.count+=self.batch_size
      yield indices

  def __len__(self):
    return len(self.labels)//self.batch_size

In [7]:
# kwargs = {'num_workers': 1, 'pin_memory': True} #if cuda else {}
train_dataloader = DataLoader(_train_dataset, batch_sampler= BatchSamplerUniform(_train_dataset.src_labels, 10, 25))#, **kwargs)
test_dataloader = DataLoader(_test_dataset, batch_sampler= BatchSamplerUniform(_test_dataset.src_labels, 10, 25))#, **kwargs)

## Define models

In [8]:
import torch
import torch.nn as nn
from torch.nn.modules.pooling import MaxPool2d

In [ ]:
# class BasicConv2d(nn.Module):
#   def __init__(self, in_channels, out_channels, kernel_size, stride ):
#     super().__init__()
#     self.model = nn.Sequential(
#         nn.Conv2d(in_channels=in_channels,out_channels=out_channels,kernel_size=kernel_size,stride=stride),
#         nn.BatchNorm2d(num_features=out_channels)
#     )
#     self.relu = nn.ReLU()

#   def forward(self,x):
#     return self.relu(self.model(x))

In [10]:
class FaceSimilarity(nn.Module):
  def __init__(self):
    super().__init__()
    # self.conv_layers = nn.Sequential(
    #         nn.Conv2d(1, 64, kernel_size=3, padding=1),
    #         nn.ReLU(),
    #         nn.MaxPool2d(kernel_size=2, stride=2),
    #         nn.Conv2d(64, 128, kernel_size=3, padding=1),
    #         nn.ReLU(),
    #         nn.MaxPool2d(kernel_size=2, stride=2),
    #         nn.Conv2d(128, 256, kernel_size=3, padding=1),
    #         nn.ReLU(),
    #         nn.Conv2d(256, 256, kernel_size=3, padding=1),
    #         nn.ReLU(),
    #         nn.MaxPool2d(kernel_size=2, stride=2),
    #         nn.Conv2d(256, 512, kernel_size=3, padding=1),
    #         nn.ReLU(),
    #         nn.Conv2d(512, 512, kernel_size=3, padding=1),
    #         nn.ReLU(),
    #         nn.MaxPool2d(kernel_size=2, stride=2),
    #         nn.Conv2d(512, 512, kernel_size=3, padding=1),
    #         nn.ReLU(),
    #         nn.Conv2d(512, 512, kernel_size=3, padding=1),
    #         nn.ReLU(),
    #         nn.MaxPool2d(kernel_size=2, stride=2)
    #     )
    self.conv_layers = nn.Sequential(nn.Conv2d(1, 32, 5), nn.PReLU(),
                                     nn.MaxPool2d(2, stride=2),
                                     nn.Conv2d(32, 64, 5), nn.PReLU(),
                                     nn.MaxPool2d(2, stride=2))

    self.fc_layers = nn.Sequential(nn.Linear(64 * 4 * 4, 256),
                        nn.PReLU(),
                        nn.Linear(256, 256),
                        nn.PReLU(),
                        nn.Linear(256,2)
                        )
    # self.conv_layers = nn.Sequential(
    #     BasicConv2d(in_channels=3,out_channels=64,kernel_size=7,stride=2),
    #     nn.MaxPool2d(kernel_size=3,stride=2),
    #     #nn.LayerNorm(55),
    #     BasicConv2d(in_channels=64,out_channels=64,kernel_size=1,stride=1),
    #     BasicConv2d(in_channels=64,out_channels=192,kernel_size=3,stride=1),
    #     #nn.LayerNorm(55),
    #     nn.MaxPool2d(kernel_size=3,stride=2),
    #     BasicConv2d(in_channels=192,out_channels=192,kernel_size=1,stride=1),
    #     BasicConv2d(in_channels=192,out_channels=384,kernel_size=3,stride=1),
    #     nn.MaxPool2d(kernel_size=3,stride=2),
    #     BasicConv2d(in_channels=384,out_channels=384,kernel_size=1,stride=1),
    #     BasicConv2d(in_channels=384,out_channels=256,kernel_size=3,stride=1),
    #     BasicConv2d(in_channels=256,out_channels=256,kernel_size=1,stride=1),
    #     BasicConv2d(in_channels=256,out_channels=256,kernel_size=3,stride=1),
    #     BasicConv2d(in_channels=256,out_channels=256,kernel_size=1,stride=1),
    #     BasicConv2d(in_channels=256,out_channels=256,kernel_size=3,stride=1),
    #     nn.MaxPool2d(kernel_size=3,stride=2),
    # )

    # self.fc_layers = nn.Sequential(
    #     nn.Linear(in_features=2*2*256,out_features=128),
    #     nn.Linear(in_features=128, out_features=128),
    #     #nn.Linear(in_features=32*128, out_features=1*128)
    # )

  def forward(self,x):
    conv_output = self.conv_layers(x)
    conv_output = conv_output.view(conv_output.shape[0],-1)
    linear_layers_output  = self.fc_layers(conv_output)
    l2_norm_output  =(linear_layers_output/(linear_layers_output.pow(2).sum(1, keepdim=True).sqrt())).squeeze(0)
    return l2_norm_output

  def get_embedding(self,x):
    return self.forward(x)

In [11]:
class TripletModel(nn.Module):
    def __init__(self):
      super().__init__()
      self.face_similarity = FaceSimilarity()
    
    def forward(self, anchor, positive, negative):
      rep_1 = self.face_similarity(anchor)
      rep_2 = self.face_similarity(positive)
      rep_3 = self.face_similarity(negative)
      return rep_1, rep_2, rep_3

Define metrics

In [13]:
class TripletLoss(nn.Module):
  def __init__(self, alpha):
    super().__init__()
    self.margin = alpha

  def forward(self, embeddings, target_labels):
    distance_positive = (embeddings[:,0] - embeddings[:,1]).pow(2).sum(1)
    distance_negative = (embeddings[:,0] - embeddings[:,2]).pow(2).sum(1)
    loss = torch.nn.functional.relu(distance_positive - distance_negative + self.margin)
    accuracy = 100*(1-torch.count_nonzero(loss)/len(loss))
    return loss.mean(), accuracy

In [14]:
from itertools import combinations

class OnlineTripletLoss(nn.Module):
  def __init__(self, alpha):
    super().__init__()
    self.margin = alpha

  def pdist(self, vectors):
    distance_matrix = -2 * vectors.mm(torch.t(vectors)) + vectors.pow(2).sum(dim=1).view(1, -1) + vectors.pow(2).sum(
        dim=1).view(-1, 1)
    #distance_matrix = np.linalg.norm(vectors-b)
    return distance_matrix

  def hardest_negative(self, loss_values):
    hard_negative = np.argmax(loss_values)
    return hard_negative if loss_values[hard_negative] > 0 else None

  def get_pairs_with_hard_negative(self, embeddings, target_labels):
    distance_matrix = self.pdist(embeddings)
    distance_matrix = distance_matrix.cpu()
    labels = target_labels.cpu().data.numpy()
    triplets = []

    for label in set(labels.flatten()):
        label_mask = (labels == label)
        label_indices = np.where(label_mask)[0]
        if len(label_indices) < 2:
            continue
        negative_indices = np.where(np.logical_not(label_mask))[0]
        anchor_positives = list(combinations(label_indices, 2))  # All anchor-positive pairs
        anchor_positives = np.array(anchor_positives)

        ap_distances = distance_matrix[anchor_positives[:, 0], anchor_positives[:, 1]]
        for anchor_positive, ap_distance in zip(anchor_positives, ap_distances):
            loss_values = ap_distance - distance_matrix[torch.LongTensor(np.array([anchor_positive[0]])), torch.LongTensor(negative_indices)] + self.margin
            loss_values = loss_values.data.cpu().numpy()
            hard_negative = self.hardest_negative(loss_values)
            if hard_negative is not None:
                hard_negative = negative_indices[hard_negative]
                triplets.append([anchor_positive[0], anchor_positive[1], hard_negative])

    if len(triplets) == 0:
        triplets.append([anchor_positive[0], anchor_positive[1], negative_indices[0]])

    triplets = np.array(triplets)

    return torch.LongTensor(triplets)

  def forward(self, embeddings, target_labels):
    embeddings = torch.cat(embeddings,dim=1)
    for sub_list in target_labels:
      target_labels = torch.stack(sub_list,dim=0).T
    triplets = self.get_pairs_with_hard_negative(embeddings, target_labels)
    # if embeddings.is_cuda:
    #         triplets = triplets.cuda()
    ap_distances = (embeddings[triplets[:, 0]] - embeddings[triplets[:, 1]]).pow(2).sum(1)  # .pow(.5)
    an_distances = (embeddings[triplets[:, 0]] - embeddings[triplets[:, 2]]).pow(2).sum(1)  # .pow(.5)
    loss = torch.nn.functional.relu(ap_distances - an_distances + self.margin)
    accuracy = 100*torch.count_nonzero(loss)/len(loss)

    return loss.mean(), accuracy

In [15]:
def train_model(dataloader,model,optimizer,criterion):
  model.train()
  train_loss=0
  classification_accuracy=0
  for i, (input, target) in enumerate(dataloader):
    optimizer.zero_grad()
    output = model(*input)#.cuda()
    if type(output) not in (tuple, tuple):
       output = (output,)
    loss_inputs = output
    if target is not None:
       target = (target,)
       loss_inputs += target
    loss, accuracy = criterion(output,target)
    classification_accuracy+=accuracy
    loss.backward()
    train_loss+=loss
    optimizer.step()
  return train_loss/len(dataloader), classification_accuracy/len(dataloader)

In [16]:
def eval_model(dataloader,model,optimizer,criterion):
  model.eval()
  val_loss=0
  classification_accuracy=0
  with torch.no_grad():
    for i, (input, target) in enumerate(dataloader):
      output = model(*input)#.cuda()
      if type(output) not in (tuple, tuple):
        output = (output,)
      loss_inputs = output
      if target is not None:
       target = (target,)
       loss_inputs += target
      loss, accuracy = criterion(output,target)
      classification_accuracy+=accuracy
      val_loss+=loss
    return val_loss/len(dataloader), classification_accuracy/len(dataloader)

In [17]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [18]:
def fit(max_epochs, model, optimizer, criterion):
  min_val_loss = float('inf')

  for i in range(max_epochs):
    start_time = time.time()

    # train model
    train_loss, train_accuracy = train_model(train_dataloader, model, optimizer, criterion)
    
    # val model loss
    val_loss, val_accuracy = eval_model(test_dataloader, model, optimizer, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {i:02} | Time: {epoch_mins}m {epoch_secs}s')
    print("Epoch {} elapsed, train_accuracy: {}, val_accuracy: {} , train_loss: {}, val_loss: {}".format(i,train_accuracy,val_accuracy,train_loss,val_loss))
    if val_loss<min_val_loss:
      min_val_loss = val_loss
    #  torch.save(model, '/content/face_verification.pt')

In [20]:
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = OnlineTripletLoss(alpha=1)
model = TripletModel().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3, weight_decay=1e-4)
print(sum(p.numel() for p in model.parameters() if p.requires_grad))
fit(10, model, optimizer, criterion)

380806
Epoch: 00 | Time: 12m 9s
Epoch 0 elapsed, train_accuracy: 99.58333587646484, val_accuracy: 97.5 , train_loss: 0.9997555613517761, val_loss: 0.9750069379806519
Epoch: 01 | Time: 12m 8s
Epoch 1 elapsed, train_accuracy: 99.58333587646484, val_accuracy: 97.5 , train_loss: 0.9958370923995972, val_loss: 0.9750038385391235
Epoch: 02 | Time: 11m 50s
Epoch 2 elapsed, train_accuracy: 99.58333587646484, val_accuracy: 97.5 , train_loss: 0.995834469795227, val_loss: 0.9750029444694519
Epoch: 03 | Time: 11m 45s
Epoch 3 elapsed, train_accuracy: 99.58333587646484, val_accuracy: 97.5 , train_loss: 0.995834231376648, val_loss: 0.9750021696090698
Epoch: 04 | Time: 11m 50s
Epoch 4 elapsed, train_accuracy: 99.58333587646484, val_accuracy: 97.5 , train_loss: 0.9958340525627136, val_loss: 0.9750018119812012
Epoch: 05 | Time: 11m 51s
Epoch 5 elapsed, train_accuracy: 99.58333587646484, val_accuracy: 97.5 , train_loss: 0.9958335757255554, val_loss: 0.9750014543533325
Epoch: 06 | Time: 11m 58s
Epoch 6 ela

KeyboardInterrupt: ignored

In [ ]:
# saved_model = TripletModel()
# saved_model.load_state_dict(torch.load('/content/face_verification.pt'))

In [ ]:
eval_model(test_dataloader)